In [1]:
import pickle
import pandas as pd
import os
import sys
sys.path
from scipy import stats
sys.path.append('../simulation')
import TS_personal_params_pooled as pp
import TS_global_params_pooled as gtp
import numpy as np

In [3]:
with open('../distributions_rl4rl/time_to_active_participants_pop_32_short_staggered_18.pkl','rb') as f:
    t = pickle.load(f)

In [4]:
seen={}
seenset=set([])
for k,v in t.items():
    for person in v:
        if person not in seenset:
            seenset.add(person)
            if k not in seen:
                seen[k]=[]
            seen[k].append(person)

In [5]:
keys = sorted(seen)
ns = {i:seen[keys[i]] for i in range(len(keys))}

In [6]:
ns

{0: [0, 1, 2, 3, 4],
 1: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 2: [15, 16, 17, 18, 19],
 3: [20, 21, 22, 23],
 4: [24, 25, 26, 27],
 5: [28, 29, 30, 31]}

In [110]:
def get_interventions(results_path,pop_size,update_time,case,phrase,ophrase):
    
    
    files = os.listdir('{}{}'.format(results_path,case))
    
    steps = []

    #check population and pop_size,update_time
    files = [f for f in files if '.DS' not in f and pop_size in f and phrase in f and ophrase not in f and 'update_days_{}'.format(update_time) in f  and '2015' not in f    ]
    #files = files[:50]
    print(len(files))
    #print(files)
    seen = 0 
    to_return={}
    to_return_o={}
    gids = {}
    for ff in files:
        #sim = int(ff.split('sim_')[1].strip('_continuous_pretreatment_base_coef_scale_hybrid.pkl'))
        #try:
        with open('{}{}/{}'.format(results_path,case,ff),'rb') as f:
            d = pickle.load(f)
            #print(d['gids'])
            #return d
            #print(d.keys())
            if len(d)>0:
            #if int(sim)<60:
                for k,v in d['history'].items():
                    #print(v.keys())
                    #print(d.keys())
                    #print(d['actions'])
                    #print(k.split('-'))
                    gid = int(k.split('-')[0])
                    rgid = int(k.split('-')[-1])
                    if gid not in to_return:
                        to_return[gid]=[]
                    if gid not in to_return_o:
                        to_return_o[gid]=[]
                    gids[gid]=rgid
                    if v['avail'] and v['decision_time']:
                        to_return[gid].append(v['action'])
                        
                        to_return_o[gid].append(v['other_action'])
               
                #print(len([k['steps'] for k in d['history'].values() if k['avail'] and k['decision_time']]))
        
    return to_return,to_return_o,gids



In [22]:
def get_regret(p):
    to_return={}
    for time,dicts in p.items():
        for person,sims in dicts.items():
            if person not in to_return:
                to_return[person]={}
            for sim,r in sims.items():
                if sim not in to_return[person]:
                    to_return[person][sim]=[]
                to_return[person][sim].append(r)
                
    for person in to_return:
        for sim in to_return[person]:
            to_return[person][sim]=sum(to_return[person][sim])
    return to_return

In [28]:
t= get_interventions('../../Downloads/pooling_results_testing/personalized/','32',7,'case_two','burden816747time_cond','inc')
#r = get_regret(t)

50


In [29]:
#r = get_regret(t)

t[2]

{0: 2,
 1: 2,
 2: 2,
 3: 1,
 4: 1,
 5: 1,
 6: 1,
 7: 1,
 8: 1,
 9: 2,
 10: 1,
 11: 2,
 12: 1,
 13: 2,
 14: 1,
 15: 1,
 16: 1,
 17: 1,
 18: 2,
 19: 2,
 20: 2,
 21: 1,
 22: 1,
 23: 2,
 24: 1,
 25: 2,
 26: 1,
 27: 2,
 28: 2,
 29: 2,
 30: 1,
 31: 2}

In [34]:
def by_cohort(cohorts,regrets,gids):
    to_return = {}
    gs = {}
    for k,v in cohorts.items():
        temp = []
        tempg=[]
        for person in v:
            reg = regrets[person]
            temp.extend(reg)
            tempg.append(gids[person])
        to_return[k]=temp
        gs[k]=tempg
    return to_return,gs

In [36]:
checkbatch = by_cohort(ns,t[0],t[2])
checktrue = by_cohort(ns,t[1],t[2])
#checkbatch = by_cohort(ns,r)
for k,v in checkbatch[0].items():
    print(k)
    print(np.array(v).mean())
    print(np.array(checktrue[0][k]).mean())
    print(np.array(checktrue[1][k]).mean())

0
0.5894736842105263
0.44903674280039724
1.6
1
0.6255314960629921
0.7068602362204724
1.3
2
0.6051851851851852
0.6213213213213213
1.4
3
0.5869221260815822
0.5338689740420272
1.5
4
0.6149174078780177
0.5605336721728081
1.5
5
0.5633893919793014
0.3516946959896507
1.75


In [112]:
t= get_interventions('../../Downloads/pooling_results_testing/batch/','32',7,'case_three','burden818twotimesaveshortmajoritydifmore','inc')
checkbatch = by_cohort(ns,t[0],t[2])
checktrue = by_cohort(ns,t[1],t[2])
#checkbatch = by_cohort(ns,r)
checks=[]
for k,v in checkbatch[0].items():
    print(k)
    print(np.array(v).mean())
    print(np.array(checktrue[0][k]).mean())
    print(np.array(checktrue[1][k]).mean())
    print(np.array(checktrue[0][k]).mean()-np.array(v).mean())
    checks.append(abs(np.array(checktrue[0][k]).mean()-np.array(v).mean()))
print(np.array(checks).sum())

50
0
0.5763636363636364
0.3447952047952048
1.6
-0.2315684315684316
1
0.5780528375733855
0.6018003913894325
1.3
0.023747553816046985
2
0.5663927855711423
0.3180761523046092
1.4
-0.24831663326653303
3
0.5199506172839506
0.38306172839506175
1.5
-0.1368888888888889
4
0.5146734955185659
0.7172855313700384
1.5
0.20261203585147247
5
0.49683269476372927
0.548786717752235
1.75
0.05195402298850571
0.8950875663798786


In [103]:
t= get_interventions('../../Downloads/pooling_results_testing/batch/','32',7,'case_three','burden818twotimesaveshortmajoritydif','?')
checkbatch = by_cohort(ns,t[0],t[2])
checktrue = by_cohort(ns,t[1],t[2])
#checkbatch = by_cohort(ns,r)
checks=[]
avs=[]
sent = []
for k,v in checkbatch[0].items():
    print(k)
    print(np.array(v).mean())
    sent.extend(v)
    print(np.array(checktrue[0][k]).mean())
    print(np.array(checktrue[1][k]).mean())
    print(np.array(checktrue[0][k]).mean()-np.array(v).mean())
    checks.append(abs(np.array(checktrue[0][k]).mean()-np.array(v).mean()))
    avs.extend(checktrue[0][k])
print(np.array(checks).sum())
print(np.array(sent).mean())

50
0
0.5938261738261739
0.49006993006993005
1.6
-0.10375624375624382
1
0.6040802348336595
0.7324755381604696
1.3
0.12839530332681015
2
0.5990781563126253
0.4264729458917836
1.4
-0.1726052104208417
3
0.559037037037037
0.4716543209876543
1.5
-0.08738271604938269
4
0.5616645326504481
0.9189244558258642
1.5
0.3572599231754161
5
0.545772669220945
0.7548659003831417
1.75
0.2090932311621967
1.058492627890891
0.5837400654511454


In [74]:
56

0.5351198377581121

In [109]:
with open('../../Downloads/actions_burden818twotimesaveshortmajoritydifmore.pkl','rb') as f:
    tt = pickle.load(f)
t=tt['case_three']
checkbatch = by_cohort(ns,t[0],t[2])
checktrue = by_cohort(ns,t[1],t[2])
avs = []
#checkbatch = by_cohort(ns,r)
checks = []
sent=[]
for k,v in checkbatch[0].items():
    print(k)
    sent.extend(v)
    print(np.array(v).mean())
    print(np.array(checktrue[0][k]).mean())
    print(np.array(checktrue[1][k]).mean())
    print(np.array(checktrue[0][k]).mean()-np.array(v).mean())
    avs.extend(checktrue[0][k])
    checks.append(abs(np.array(checktrue[0][k]).mean()-np.array(v).mean()))
print(np.array(checks).sum())
print(np.array(sent).mean())

0
0.4627772227772228
0.4911288711288711
1.6
0.02835164835164833
1
0.4785029354207436
0.7333659491193738
1.3
0.2548630136986302
2
0.4143486973947896
0.4286172344689379
1.4
0.014268537074148324
3
0.4008888888888889
0.4755308641975309
1.5
0.07464197530864197
4
0.4677592829705506
0.918898847631242
1.5
0.4511395646606914
5
0.43818646232439334
0.752618135376756
1.75
0.3144316730523627
1.1376964121461228
0.45004830917874394


In [84]:
t= get_interventions('../../Downloads/pooling_results_testing/personalized/','32',7,'case_two','None818two','inc')
checkbatch = by_cohort(ns,t[0],t[2])
checktrue = by_cohort(ns,t[1],t[2])
#checkbatch = by_cohort(ns,r)
checks=[]
avs=[]
sent=[]
for k,v in checkbatch[0].items():
    print(k)
    sent.extend(v)
    print(np.array(v).mean())
    print(np.array(checktrue[0][k]).mean())
    print(np.array(checktrue[1][k]))
    print(np.array(checktrue[1][k]).mean())
    print(abs(np.array(checktrue[0][k]).mean()-np.array(v).mean()))
    avs.extend(checktrue[0][k])
    checks.append(abs(np.array(checktrue[0][k]).mean()-np.array(v).mean()))
print(np.array(checks).sum())
print(np.array(sent).mean())

51
0
0.562785348743603
0.4158357771260997
[2 2 2 1 1]
1.6
0.14694957161750327
1
0.6127892598480834
0.6780062795243061
[1 1 1 1 2 1 2 1 2 1]
1.3
0.06521701967622273
2
0.5877482867777507
0.5960390069688432
[1 1 1 2 2]
1.4
0.008290720191092449
3
0.5739141586867305
0.5103311673506612
[2 1 1 2]
1.5
0.06358299133606926
4
0.5883842898861672
0.5062131235472913
[1 2 1 2]
1.5
0.08217116633887589
5
0.5429544069333572
0.2991273840847939
[2 2 1 2]
1.75
0.24382702284856328
0.6100384920083268
0.5846437763896118


In [55]:
t= get_interventions('../../Downloads/pooling_results_testing/batch/','32',7,'case_two','None818','inc')
checkbatch = by_cohort(ns,t[0],t[2])
checktrue = by_cohort(ns,t[1],t[2])
#checkbatch = by_cohort(ns,r)
checks=[]
for k,v in checkbatch[0].items():
    print(k)
    print(np.array(v).mean())
    print(np.array(checktrue[0][k]).mean())
    print(np.array(checktrue[1][k]).mean())
    print(abs(np.array(checktrue[0][k]).mean()-np.array(v).mean()))
    checks.append(abs(np.array(checktrue[0][k]).mean()-np.array(v).mean()))
print(np.array(checks).sum())

50
0
0.5424281524926686
0.4403284457478006
1.6
0.10209970674486801
1
0.5359197907585004
0.69521650682941
1.3
0.15929671607090967
2
0.5331756357185098
0.6212891780011828
1.4
0.08811354228267299
3
0.5301162790697674
0.5284302325581396
1.5
0.0016860465116278478
4
0.5277811550151976
0.5354863221884498
1.5
0.007705167173252203
5
0.516194086429113
0.3333737680060652
1.75
0.1828203184230478
0.5417214972063785


In [22]:
t= get_interventions('../../Downloads/pooling_results_testing/batch/','32',7,'case_three','burden817731','inc')
checkbatch = by_cohort(ns,t[0])
checktrue = by_cohort(ns,t[1])
#checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())
    print(np.array(checktrue[k]).mean())

51
0
0.6160948193285555
0.644496762801648
1
0.626173319023526
0.7852961051202989
2
0.618111149785358
0.5204316045945006
3
0.6182095205547724
0.5524513363805944
4
0.6262535548570574
0.9622960634635533
5
0.6203850909733263
0.8103397515162221


In [79]:
t= get_results_new_save_person('../../Downloads/pooling_results_testing/hob_clipped/','32',7,'case_one','burden817731','inc')
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
39.89120000000001
1
40.1758
2
39.13459999999999
3
39.0105
4
36.780750000000005
5
37.34375


In [80]:
t= get_results_new_save_person('../../Downloads/pooling_results_testing/hob_clipped/','32',7,'case_two','burden817731','inc')
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
37.969800000000006
1
38.65089999999999
2
37.04579999999999
3
36.618500000000004
4
33.86125
5
33.88075


In [81]:
t= get_results_new_save_person('../../Downloads/pooling_results_testing/hob_clipped/','32',7,'case_three','burden817731','inc')
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
39.35122467712511
1
51.40097696795171
2
35.7128585044465
3
50.26213872618795
4
41.86718708297336
5
42.24147302469068


In [82]:
t= get_results_new_save_person('../../Downloads/pooling_results_testing/personalized/','32',7,'case_two','burden817731','inc')
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
44.41660000000003
1
42.650600000000004
2
42.46820000000002
3
43.229750000000024
4
41.22400000000002
5
44.77775000000003


In [83]:
t= get_results_new_save_person('../../Downloads/pooling_results_testing/personalized/','32',7,'case_one','burden817731','inc')
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
37.551800000000014
1
38.54310000000001
2
38.2934
3
38.209250000000004
4
34.943000000000005
5
36.87225000000001


In [84]:
t= get_results_new_save_person('../../Downloads/pooling_results_testing/personalized/','32',7,'case_three','burden817731','inc')
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
40.16774222776907
1
51.92544741193913
2
42.09993268056138
3
54.46384500485858
4
42.634115429602744
5
42.585435238668175


In [61]:
with open('../../Downloads/regret_case_two_816747_person_dict.pkl','rb') as f:
    t = pickle.load(f)
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
21.745600000000007
1
23.173500000000008
2
24.748000000000008
3
23.859500000000004
4
22.418750000000003
5
20.159000000000002


In [62]:
with open('../../Downloads/regret_case_one_816747_person_dict.pkl','rb') as f:
    t = pickle.load(f)
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
18.397400000000005
1
19.87680000000001
2
19.95180000000001
3
21.617000000000008
4
19.92750000000001
5
22.53550000000001


In [63]:
with open('../../Downloads/regret_case_three_816747_person_dict.pkl','rb') as f:
    t = pickle.load(f)
r = get_regret(t)
checkbatch = by_cohort(ns,r)
for k,v in checkbatch.items():
    print(k)
    print(np.array(v).mean())

0
20.31489657902877
1
33.41000131562465
2
21.584929740941437
3
34.48508551484087
4
31.67013433264591
5
28.544910313535265
